# Finetuning MobileNetV2

In [ ]:
#imports
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v2
import time

## Data Preprocessing

In [ ]:
#loading dataset
drive.mount('/content/drive')
wd = "/content/drive/MyDrive/Colab Notebooks/stat453/project" #change
os.chdir(wd)
data_dir = './RealWaste'
data = datasets.ImageFolder(data_dir,transform=None)
print(f'dataset size: {len(data)}')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset size: 4752


In [ ]:
#train-val-test split
train_data, val_data, test_data = random_split(data,[int(len(data)*.8),int(len(data)*.1),len(data)-int(len(data)*.8)-int(len(data)*.1)])
print(f'training size: {len(train_data)}')
print(f'validation size: {len(val_data)}')
print(f'testing size: {len(test_data)}')

training size: 3801
validation size: 475
testing size: 476


In [ ]:
#transformations

#training images transformation and data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#testing images transformation(no augmentation)
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#class to transform datasets
class SubsetTransform(Dataset):
  def __init__(self,subset,transform=None):
    self.subset = subset
    self.transform = transform
  def __getitem__(self,idx):
    image,label = self.subset[idx]
    if self.transform:
      image = self.transform(image)
    return image,label
  def __len__(self):
    return len(self.subset)

#create new transformed datasets
train_dataset = SubsetTransform(train_data,train_transform)
val_dataset = SubsetTransform(val_data,test_transform)
test_dataset = SubsetTransform(test_data,test_transform)

In [ ]:
#dataloaders
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)

## Fine Tuning

In [ ]:
num_classes = len(data.classes)

#load pretrained mobilenet model
model = mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel,num_classes) #adapt last layer to our number of classes
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 133MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
#training

train_acc_lst, valid_acc_lst, train_loss_lst, valid_loss_lst = [], [], [], []
start_time = time.time()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
num_epochs = 30
for epoch in range(num_epochs):
  model.train()
  total_loss = 0.0
  correct = 0
  total = 0
  for images,labels in train_loader:
    images,labels = images.to(device),labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs,labels)
    loss.backward()
    _,predicted = torch.max(outputs,1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)
    optimizer.step()
    total_loss += loss.item()
  epoch_loss = total_loss/len(train_loader)
  epoch_acc = correct/total

  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0
  with torch.no_grad():
    for images,labels in val_loader:
      images,labels = images.to(device),labels.to(device)
      outputs = model(images)
      loss = criterion(outputs,labels)
      val_loss += loss.item()
      _,predicted = torch.max(outputs,1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)

  epoch_val_loss = val_loss/len(val_loader)
  epoch_val_acc=correct/total
  train_acc_lst.append(epoch_acc)
  valid_acc_lst.append(epoch_val_acc)
  train_loss_lst.append(epoch_loss)
  valid_loss_lst.append(epoch_val_loss)

  print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {epoch_val_acc:.4f}')

tot_time = (time.time() - start_time)/60

In [ ]:
        save_path = os.path.join(save_dir, f"model_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"Model saved at {save_path}")
